## Projet GPS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from pyroutelib3 import Router
import folium
from time import time
import os

os.chdir("..")

from ipywidgets import interact

from src.models import path_PCC
from src.tools import Correc_Etiq_bis, nettoyage

os.chdir("data")

### Motivation

Maintenant que nous avons testé tous nos algorithmes nous trouvions intéressant de venir se placer dans un cadre concret et quoi de plus naturel que de se placer dans le graphe du réseau routier français. Quand on pense au plus court chemin on pense souvent en termes de déplacement et à l'approche des vacances on peut ainsi essayer de trouver les meilleurs trajets reliant certaines grandes villes française, sous certaines contraintes comme la consommation de carburant, le temps de trajet ou encore le prix des péages afin de faire des économies sur le déplacement.

### Un peu de data-scrapping pour changer des graphes.

Afin de récupérer l'ensemble des données nécessaires à ce petit projet, nous avons réalisé du data-scrapping, c'est à dire récupérer des données sur internet via un algorithme. Nous vous avons fourni l'ensemble des codes qui concernent cette partie dans un autre notebook que nous avons détaillé. Pour la suite, seul les fichiers csv, que nous vous avons fournis sont utiles.

**Attention :**  Nous vous déconseillons d'exécuter le notebook qui se rapporte à cette partie, en effet ce notebook est très sensible aux mises à jour de google et nous-mêmes nous avons rencontré plusieurs problèmes de compatibilité ou encore de blocage d'adresse IP et nous ne souhaitons pas que cela vous arrive. Nous vous l'avons simplement transmis car il représente une partie importante de notre projet.

### Lecture et traitement des données

Pour ce projet nous disposons de deux fichiers de type csv.

- Le premier fichier **commune1.csv** nous a été donné lors du premier semestre au cours du séance de travaux pratiques, il contient énormément de données mais surtout les longitudes et les lattitudes de toutes les communes francaise. Cela nous servira à faire un fond de carte.

- Le second **Reseaux_routier_VF.csv** est le résultat de notre recherche sur internet

In [2]:
df_commune1 = pd.read_csv("commune1.csv")
df_GPS_2 = pd.read_table("Reseaux_routier_VF.txt", sep=",")
df_commune1 = df_commune1.fillna(0)
long = df_commune1["long"]
lat = df_commune1["lat"]
del df_GPS_2["0"]

In [3]:
ville_d = list(set(df_GPS_2["depart"]))
ville_a = list(set(df_GPS_2["arrive"]))
villes = list(set(ville_d + ville_a))
villes = list(set([v.replace(" ", "") for v in villes]))
villes = sorted(villes)

# variable contenant toutes les villes sauf GENEVE,SOISSONS,VIRTON,MONACO,SAINT-BREVIN.
# Ces villes ne sont soient pas en France, soient dures à trouver

villes1 = [ville for ville in villes if (ville != "GENEVE")]
villes1 = [ville for ville in villes1 if (ville != "SOISSONS")]
villes1 = [ville for ville in villes1 if (ville != "VIRTON")]
villes1 = [ville for ville in villes1 if (ville != "MONACO")]
villes1 = [ville for ville in villes1 if (ville != "SAINT-BREVIN")]

In [4]:
GPS = {}
for ville in villes1:
    ville = ville.replace(" ", "")
    GPS[ville] = []
    try:
        lg = float(df_commune1[df_commune1["com_nom"] == ville]["long"])
        lt = float(df_commune1[df_commune1["com_nom"] == ville]["lat"])
        GPS[ville].append(lg)
        GPS[ville].append(lt)

    # except à cause de la ville de la rochelle.
    except:
        lg = float(df_commune1[df_commune1["com_nom"] == ville]["long"].values[1])
        lt = float(df_commune1[df_commune1["com_nom"] == ville]["lat"].values[1])
        GPS[ville].append(lg)
        GPS[ville].append(lt)

# On ajoute ces villes à la main

GPS["GENEVE"] = [6.14569, 46.20222]
GPS["SOISSONS"] = [3.323420, 49.376636]
GPS["VIRTON"] = [5.533507, 49.567574]
GPS["MONACO"] = [7.424616, 43.738418]
GPS["SAINT-BREVIN"] = [-2.165807, 47.246659]

Pour pouvoir naviguer simplement entre toutes les villes, on va créer la variable **codage** de la facon suivante.

In [5]:
codage = {}
for i in range(len(villes)):
    codage[villes[i]] = i
    codage[i] = villes[i]

### Création des matrices pour le modèle Pulp

Ici avec les données que nous avons, nous allons essayer de minimiser le trajet en terme de distance en imposant par exemple des contraintes sur le péage le temps de trajet, ou encore le prix du carburant. L'ensemble de ces données ont été recueillies sur le site **via michelin**

In [6]:
N = len(villes)

C = np.zeros((N, N))
R1 = np.zeros((N, N))
R2 = np.zeros((N, N))
R3 = np.zeros((N, N))

for u in list(df_GPS_2.index):
    row = list(df_GPS_2.iloc[u])
    i = codage[row[0]]
    j = codage[row[1]]
    c = row[2]
    r1 = row[3]
    r2 = row[4]
    r3 = row[5]

    # entre deux villes, on peut effectuer un trajet dans les deux sens.
    # les sens interdits n'existent pas ici.
    C[i][j] = c
    C[j][i] = c

    # on récupère les contraintes.
    R1[i][j] = r1
    R1[j][i] = r1
    R2[i][j] = r2
    R2[j][i] = r2
    R3[i][j] = r3
    R3[j][i] = r3

R = {1: R1, 2: R2, 3: R3}  # dictionnaire des consommations de ressources.

In [7]:
df_GPS_2

,depart,arrive,distance,carbu,peage,duree
0,GENEVE,DIJON,264.00,21.09,26.7,160
1,NANTES,BREST,297.00,23.83,0.0,208
2,SAINT-BREVIN,NANTES,69.00,5.79,0.0,74
3,TARBES,TOULOUSE,158.00,12.78,8.8,113
4,TARBES,PAU,45.09,3.86,2.8,46
...,...,...,...,...,...,...
93,BESANCON,GENEVE,177.00,14.50,0.0,163
94,BORDEAUX,LIMOGES,222.00,17.98,0.0,174
95,LILLE,AMIENS,140.00,11.38,9.3,118
96,AMIENS,LAON,130.00,10.47,8.5,86


### Création du graphe correspondant via networkx

In [8]:
g_ville = nx.DiGraph()
for ind in list(df_GPS_2.index):
    l = list(df_GPS_2.iloc[ind, :])
    i = codage[l[0].replace(" ", "")]
    j = codage[l[1].replace(" ", "")]
    g_ville.add_nodes_from([i, j])

    # On ajoute les arcs dans les deux sens
    g_ville.add_edges_from([(i, j, {"weight": l[2:6]}), (j, i, {"weight": l[2:6]})])

In [9]:
Adj_ville = np.zeros((N, N))
for u, v in g_ville.edges():
    Adj_ville[u, v] = 1

### Affichage sur fond de carte

**Motivations** :

Nous avions à cœur d'utiliser nos algorithmes dans un cas concret et ainsi avoir une visualisation et un
côté pratique et ludique quant au test de nos algorithmes. Nous avons donc décidé de faire une fonction d'affichage
en utilisant un fond de carte de France et de permettre à l'utilisateur de choisir comme dans un GPS les valeurs
des contraintes souhaitées.


**Utilisation de la fonction d'affichage** :

Nous avons mis au point une fonction interactive depuis la fenêtre d'affiche.

Celle ci calcule en temps réel dès qu'une nouvelle donnée est entrée.

Les paramètres sur lesquels nous pouvons interagir sont les suivants :


- On peut choisir une ville de départ et une ville d'arrivée en sélectionnant le volet déroulant.

- On peut choisir des ressources pour les contraintes en entrant les valeurs souhaitées dans les cases correspondantes.

- Enfin, une case que l'on peut cocher ou décocher permet d'afficher la solution avec l'algorithme de correction d'étiquette
ou avec l'algorithme de pré-traitement.





Dans le cas où l'on utilise **la correction d'étiquette** mais **sans le pré-traitement**, on affiche sur la carte :

- en bleu : le graphe complet
- en rouge : la solution du problème
- La ville de départ : voiture verte
- ville d'arrivée : une voiture rouge

On donne aussi les valeurs avec le modèle Pulp et de celles avec la correction d'étiquettes.
On donne aussi le temps de calcul des deux algorithmes.

**Remarques :**

On peut cliquer sur les sommets pour faire afficher le nom de la ville.





Dans le cas où l'on applique **le pré-traitement**, on affiche avec le même principe :

- en bleu : le graphe réduit aux sommets et arcs réalisables
- en rouge : la solution du problème
- La ville de départ : voiture verte
- ville d'arrivée : une voiture rouge

**Remarques :**

Comme le graphe et la solution sont confondues, nous avons joué sous l'épaisseur des traits,
Ainsi, on peut bien voir en rouge et en bleu les arcs précédemment définis

**Commentaires :**

On remarquera que notre algorithme de pré-traitement est relativement efficace car dès que le problème est réalisable, seule
le graphe sous-jacent réduit correspond à solution de notre problème.




**Instance de Test :**

Nous vous laissons, le soin de tester sur toutes les combinaisons de villes et des contraintes que vous le voulez.

A titre indicatif, pour illustrer que la correction d'étiquette retourne bien le plus court chemin sous contrainte vous pouvez
sur le trajet RENNES-PERPIGNANT jouer sur le péage (le passer à 50 par exemple).

Mais on trouve des modification de chemin avec jouant sur d'autres contraintes sur de multiples trajets (principalement sur des chemins
qui relient des villes "éloignées")

Enfin, dans nos valeurs, nous avons récupéré les données directement sur Mappy avec un BOT. Les chemin que nous avons pris était toujours des
plus courts chemins en temps. Dès que c'était possible, le chemin passait par l'autoroute. Pour pourvoir jouer sur la contrainte péage nous avons alors
volontairement ajouté dans petite ville pour relier des plus grande ville. Cela nous a permis de relier des grandes villes sans prendre de péages.

In [10]:
def affiche_graph(g_ville, c, Lville, depart, arrive):
    list_edges = [list(i) for i in g_ville.edges()]
    list_nodes = [codage[i] for i in g_ville.nodes()]
    list_edges = [[codage[i[0]], codage[i[1]]] for i in list_edges]
    for node in list_nodes:
        if node != depart and node != arrive:
            folium.Marker(list(reversed(GPS[node])), popup=ville, weight=1).add_to(c)
    for arc in list_edges:
        routeLatLons = [list(reversed(GPS[arc[0]])), list(reversed(GPS[arc[1]]))]
        folium.PolyLine(routeLatLons, color="blue", weight=3, opacity=1).add_to(c)
    for i in Lville:
        routeLatLons.append(list(reversed(GPS[codage[i]])))
    folium.PolyLine(routeLatLons, color="red", weight=2, opacity=1).add_to(c)

In [17]:
class Pb_insoluble(Exception):
    pass


print("\t\tGPS BY MALO BAPTISTE LOIC")


@interact(
    fuel="200",
    toll="200",
    max_time="3000",
    start=villes,
    end=villes,
    with_reduction=False,
)
def affichage(
    with_reduction=False,
    fuel="200",
    toll="200",
    max_time="3000",
    start="RENNES",
    end="PERPIGNAN",
):
    # cast des bons types :
    try:
        if fuel == "" or toll == "" or max_time == "":
            raise ValueError("Entrez a reel value")
        elif "," in fuel or "," in toll or "," in max_time:
            raise ValueError("remove ',' ")
        else:
            fuel = float(fuel)
            toll = float(toll)
            max_time = float(max_time)

        # On récupère le numéro des villes.
        s = codage[start]
        t = codage[end]

        b = {1: fuel, 2: toll, 3: max_time}
        b_correc = [fuel, toll, max_time]

        print("----Solution with Pulp model ----")
        t1 = time()
        res = path_PCC(N, C, Adj_ville, s, t, R, b)
        t2 = time()

        if res["statu"] == "Infeasible":
            raise Pb_insoluble("The problem is infeasible")

        Lville = res["trace"]

        print("Length longest path : {}".format(res["consommations"][0]))
        for r in range(1, 4):
            print(
                "Resource consumption {} : {}".format(
                    r, np.round(res["consommations"][r], 3)
                )
            )

        print("Calculation time : {} s".format(np.round(t2 - t1, 3)))

        print("Path : ", end=" ")

        for v in Lville:
            print(codage[v], end="-")
        print("\n----End solution with the Pulp Model")

        d = Correc_Etiq_bis(g_ville, s, t, 3, b_correc)

        print("\n\n----Solution with Label correction----")

        print("Shortest path length : {}".format(d["pcc"][0]))
        for r in range(1, 4):
            print("Resource consumption {} : {}".format(r, np.round(d["pcc"][r], 3)))

        print("Calculation time : {} s".format(np.round(d["temps"], 3)))
        print("Path: ", end=" ")
        for v in d["trace"]:
            print(codage[v], end="-")
        print("\n---- End of label correction solution----\n\n")

        if res["statu"] == "Optimal":
            routeLatLons = []
            coor_depart = list(reversed(GPS[start]))
            coor_arrivee = list(reversed(GPS[end]))

            # on initialise la carte

            c = folium.Map(location=[46.611715, 2.768555], zoom_start=6)

            # choix de l'icone
            icone = "car"

            # on marque les villes du graph

            folium.Marker(
                coor_depart,
                popup="Start : " + start,
                icon=folium.Icon(icon=icone, prefix="fa", color="green"),
            ).add_to(c)
            folium.Marker(
                coor_arrivee,
                popup="End : " + end,
                icon=folium.Icon(icon=icone, prefix="fa", color="red"),
            ).add_to(c)
            if not with_reduction:
                for ville in villes:
                    if ville != start and ville != end:
                        folium.Marker(
                            list(reversed(GPS[ville])), popup=ville, weight=1
                        ).add_to(c)

                router = Router("car")

                for i in df_GPS_2.index:
                    city_d = df_GPS_2["depart"][i]
                    city_a = df_GPS_2["arrive"][i]
                    graph = [list(reversed(GPS[city_d])), list(reversed(GPS[city_a]))]
                    folium.PolyLine(graph, color="blue", weight=1, opacity=1).add_to(c)
                for i in Lville:
                    routeLatLons.append(list(reversed(GPS[codage[i]])))
                folium.PolyLine(routeLatLons, color="red", weight=2, opacity=1).add_to(
                    c
                )
            else:
                t1 = time()
                gp_ville = nettoyage(g_ville, s, t, b_correc, 3)["graphe"]
                t2 = time()

                print("calculation time cleaning : {} s".format(np.round(t2 - t1, 3)))
                affiche_graph(gp_ville, c, Lville, start, end)
        display(c)
    except ValueError as v1:
        print(v1)
    except Pb_insoluble as e1:
        print(e1)

		GPS BY MALO BAPTISTE LOIC


interactive(children=(Checkbox(value=False, description='with_reduction'), Text(value='200', description='fuel…